In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2
import msmrd2.tools.quaternions as quats
import msmrd2.visualization as msmrdvis
from msmrd2.potentials import patchyParticleAngular2
from msmrd2.integrators import overdampedLangevin as odLangevin

In [ ]:
# Define arbitrarily large list of rod-like particles
np.random.seed(seed=1) # seed 1 good for pentamer 
numparticles = 3
boxsize = 2.5
D = 1.0
Drot = 1.0
pyPartlist = [] 
positionList = [np.array([-1.0,0,0]), np.array([0.,0.,0.]), np.array([1.0,0.,0.]), 
                np.array([2.0,0.,0.]), np.array([3.0,0.,0.])]
orientationList = [ np.array([1.,0.,0.,0.]), np.array([np.cos(np.pi/2),0.,0.,np.sin(np.pi/2)])]

"""
# Position List for pentamer IC
th0 = 3*np.pi/5.0
th = np.pi - th0
thextra = np.pi #np.pi/2 + th/2.0
positionList = []
orientationList = []
for i in range(5):
    positionList.append(0.85*np.array([np.cos(th*i),np.sin(th*i),0.0]))
    orientation = np.array([np.cos(0.5*(th*i + thextra)),0,0,np.sin(0.5*(th*i + thextra))])
    orientationList.append(orientation)
for i in range(5):
    part = msmrd2.particle(D, Drot, positionList[i], orientationList[i])
    pyPartlist.append(part)
"""

#"""
# For an arbitrary number of particles with random positions and orientations. 
# Caution, can be caught in infinite loop if number of particles too large due to limited volume
for i in range(numparticles):
    
    overlap = True
    while overlap:
        position = np.array([boxsize*np.random.rand()-0.5*boxsize,
                             boxsize*np.random.rand()-0.5*boxsize,
                             boxsize*np.random.rand()-0.5*boxsize])
        overlap = False     
        for j in range(len(pyPartlist)):
            if np.linalg.norm(position - pyPartlist[j].position) < 1.5:#< 2.0::
                overlap = True
                continue
                
    orientation = np.array([2*np.random.rand()-1, 2*np.random.rand()-1,2*np.random.rand()-1,2*np.random.rand()-1])
    orientation = orientation/np.linalg.norm(orientation)
    #orientation = np.array([1.,0.,0.,0.])
    #orientation = np.array([np.cos(np.pi/4.0), np.sin(np.pi/4), 0.0, 0.0]) #Rotate 90 degrees in x axis
    part = msmrd2.particle(D, Drot, position, orientation)
    pyPartlist.append(part)
    print("Set particle ", i+1, " of ", numparticles, end="\r")
#"""

In [ ]:
# Create list of particles that can be read from msmrd
# Note the particles in this list will be independent from the python list.
partlist = msmrd2.integrators.particleList(pyPartlist)

In [ ]:
# Over-damped Langevin integrator definition
dt = 0.0001 #0.00001 #0.000005
seed = -1 #1 #-1 # Negative seed, uses random device as seed
bodytype = 'rigidbody'
integrator = odLangevin(dt, seed, bodytype) 
#integrator.setKbT(0.1)

In [ ]:
# Define boundary (choose either spherical or box)
# sphericalBoundary = msmrd2.sphere(radius,'reflective')
boxBoundary = msmrd2.box(boxsize,boxsize,boxsize,'reflective')
integrator.setBoundary(boxBoundary)

In [ ]:
# Define Patchy Particle potential
sigma = 1.0
strength = 160 #100# 60 #200.0
angularStrength = 20 #10 #200.0
angleDiff = 3*np.pi/5.0
patch1 = np.array([np.cos(angleDiff/2),np.sin(angleDiff/2),0.])
patch2 = np.array([np.cos(-angleDiff/2),np.sin(-angleDiff/2),0.])
#patch2 = np.array([-1.,0.,0.])
patchesCoordinates = [patch1, patch2]
potentialPatchyParticleAngular2 = patchyParticleAngular2(sigma, strength, angularStrength, patchesCoordinates)
integrator.setPairPotential(potentialPatchyParticleAngular2)

In [ ]:
#Integrate particle list and print only positions 
timesteps = 100000 #1000000 #20000000
stride = 25 #250 #1000
datafile  = open('../../data/vmd/patchyParticlesMultimer.xyz', 'w')
for i in range(timesteps):
    if i%stride == 0:
        datafile.write(str(3*len(partlist)) + '\n')
        datafile.write(str(0) + '\n')
    for j, part in enumerate(partlist):
        if i%stride == 0:
            v0 = part.position
            v1 = v0 + 0.5*sigma*quats.rotateVec(patchesCoordinates[0], part.orientation)
            v2 = v0 + 0.5*sigma*quats.rotateVec(patchesCoordinates[1], part.orientation)
            datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
            datafile.write('type_1' + ' ' + ' '.join(map(str, v1)) + '\n')
            datafile.write('type_1' + ' ' + ' '.join(map(str, v2)) + '\n')
    integrator.integrate(partlist)
    if i%10000 == 0:
        print("Percentage complete: ", 100*i/timesteps, "%", end="\r")
datafile.close()
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_patchyParticles(numparticles = numparticles, 
                                    outfname = "patchyParticlesMultimer", 
                                    tclfname = "../../data/vmd/patchyParticlesMultimer2vmd.tcl")
print("Percentage complete: ", 100, " %")

In [ ]:
# Trimer formation verification test
# Create dummy trajectory to extract bound states bewteen pairs of particles.
dummyTraj = msmrd2.trajectories.patchyDimer2(2,1)
# Require more lax tolerances since trimer bends the natural binding angle 
# (can be tested while plotting with vmd).
dummyTraj.setTolerances(0.50, 0.5*2*np.pi)
# All this bound states between two molecules corespond to be bound in a U-shape.
boundStates = [1, 2, 3, 4]
# Check the 3 bindings of the trimer
binding1 = dummyTraj.getState(partlist[0], partlist[1])
binding2 = dummyTraj.getState(partlist[0], partlist[2])
binding3 = dummyTraj.getState(partlist[1], partlist[2])
if (binding1 in boundStates) and (binding2 in boundStates) and (binding3 in boundStates):
    print("Trimer is formed")
else:
    print("No trimer formed yet")

In [ ]:
print(binding1,binding2,binding3)

In [ ]:
print(partlist[0].position,partlist[2].position,partlist[2].position)